# Rodrigo_Mesquita_DR3_TP3
## Desenvolvimento Front-End com Python (com Streamlit) [24E3_1]

streamlit run app.py --server.port=8086 --browser.serverAddress='177.192.12.238'

### 1. Explicação do Objetivo e Motivação:
Explique o objetivo do dashboard que você está desenvolvendo e a motivação por trás da escolha dos dados e funcionalidades que serão implementados.

Base: Chegada mensal de turistas pelo Rio de Janeiro, por via Aérea, segundo continentes e países de residência permanente, entre 2006-2019
Disponível em: https://www.data.rio/documents/a6c6c3ff7d1947a99648494e0745046d/about

- Objetivo: Apresentar a dados de turismo no Rio de Janeiro.
- Motivador: Conseguir identificar o crescimento ou redução da quantidade de turistas de acordo com o país de origem.
- Funcionalidades: 
    - Filtros por período, continente e país
    - Gráfico de linha exibindo tendências
    - Gráfico de pizza com número de turistas por continente
    - Gráfico de pareto com número absoluto de turistas por país
    - Histograma por mês
    - Mapa de calor por mês e país
    - Mapa de rotas

### 2. Realizar Upload de Arquivo CSV:
Crie uma interface em Streamlit que permita ao usuário fazer o upload de um arquivo CSV contendo dados de turismo do portal Data.Rio.

In [221]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    st.dataframe(df) 


Overwriting app.py


### 3. Filtro de Dados e Seleção:
Implemente seletores (radio, checkbox, dropdowns) na interface que permitam ao usuário filtrar os dados carregados e selecionar as colunas ou linhas que deseja visualizar.

In [36]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    st.DataFrame(df) 
    


Overwriting app.py


### 4. Desenvolver Serviço de Download de Arquivos:
Implemente um serviço que permita ao usuário fazer o download dos dados filtrados em formato CSV diretamente pela interface da aplicação.

In [239]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    #Mostro a tabela filtrada
    st.dataframe(df) 

    #Converto para CSV
    csv = df.to_csv().encode("utf-8")

    #Crio botão para baixar o arquivo csv
    st.download_button(
        label="Baixar Dados",
        data=csv,
        file_name="base.csv",
        mime="text/csv",
    )


Overwriting app.py


### 5. Utilizar Barra de Progresso e Spinners:
Adicione uma barra de progresso e um spinner para indicar o carregamento dos dados enquanto o arquivo CSV é processado e exibido na interface.

### Adicionei o spinner e a barra de progresso na etapa de upload 

In [270]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time


df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    #Adicionando o Spinner
    with st.spinner('Upload em andamento...'):
        time.sleep(2)
        st.success("Upload concluído!")

    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    #Adicionando a barra de progresso
    my_bar = st.progress(0, text="Carregando a Visualização...")
    for percent_complete in range(100):
        time.sleep(0.005)
        my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
    time.sleep(0.5)
    my_bar.empty()
    
    #Mostro a tabela filtrada
    st.dataframe(df) 

    #Converto para CSV
    csv = df.to_csv().encode("utf-8")

    #Crio botão para baixar o arquivo csv
    st.download_button(
        label="Baixar Dados",
        data=csv,
        file_name="base.csv",
        mime="text/csv",
    )

    

Overwriting app.py


### 6. Utilizar Color Picker:
Adicione um color picker à interface que permita ao usuário personalizar a cor de fundo do painel e das fontes exibidas na aplicação.

In [ ]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time

#Adicionando o Color Picker
col1, col2 = st.columns(2)

with col1:
    cor_fundo = st.color_picker("Escolha a cor de fundo", "#FFFFFF")  

with col2:
    cor_fonte = st.color_picker("Escolha a cor da fonte", "#000000")  

#Alterando o estilo da classe stApp
st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)


df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    #Adicionando o Spinner
    with st.spinner('Upload em andamento...'):
        time.sleep(2)
        st.success("Upload concluído!")

    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    #Adicionando a barra de progresso
    my_bar = st.progress(0, text="Carregando a Visualização...")
    for percent_complete in range(100):
        time.sleep(0.005)
        my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
    time.sleep(0.5)
    my_bar.empty()

    #Mostro a tabela filtrada
    st.dataframe(df) 

    #Converto para CSV
    csv = df.to_csv().encode("utf-8")

    #Crio botão para baixar o arquivo csv
    st.download_button(
        label="Baixar Dados",
        data=csv,
        file_name="base.csv",
        mime="text/csv",
    )


    

### 7. Utilizar Funcionalidade de Cache:
Utilize a funcionalidade de cache do Streamlit para armazenar os dados carregados de grandes arquivos CSV, evitando a necessidade de recarregá-los a cada nova interação.

In [28]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time

def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()
            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filt_e_exibe(df):
    if df is not None:
        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Pego o id da opção "Todos"
        default_cont = list_cont.index('Todos')

        #Crio o filtro de continentes
        filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=default_cont)

        #Aplico a condição para só filtrar se houver seleção
        if filter_cont != 'Todos':
            df = df.loc[df['Continente'] == filter_cont]

        #Crio o filtro de colunas
        list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, df.columns)
        #Aplico o filtro
        df = df[list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()

        #Mostro a tabela filtrada
        st.dataframe(df) 



################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

if files:
    df, csv = tratamento(files)
    filt_e_exibe(df)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")

end = time.time()
st.write(f'Tempo de execução: {end - start:.2f} segundos')
    


Overwriting app.py


### 8. Persistir Dados Usando Session State:
Implemente a persistência de dados na aplicação utilizando Session State para manter as preferências do usuário (como filtros e seleções) durante a navegação.

In [42]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time

def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()
            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filt_e_exibe(df):
    if df is not None:
        
        #Inicializo o session_state
        if 'filter_cont' not in st.session_state:
            st.session_state.filter_cont = 'Todos'
        if 'list_colunas' not in st.session_state:
            st.session_state.list_colunas = df.columns.tolist()

        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Crio o filtro de continentes
        st.session_state.filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=list_cont.index(st.session_state.filter_cont))

        #Aplico a condição para só filtrar se houver seleção
        if st.session_state.filter_cont != 'Todos':
            df = df.loc[df['Continente'] == st.session_state.filter_cont]

        #Crio o filtro de colunas
        st.session_state.list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, st.session_state.list_colunas)

        #Aplico o filtro
        df = df[st.session_state.list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()

        #Mostro a tabela filtrada
        st.dataframe(df) 



################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

if files:
    df, csv = tratamento(files)
    filt_e_exibe(df)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")

end = time.time()
st.write(f'Tempo de execução: {end - start:.2f} segundos')
    

Overwriting app.py


### 9. Criar Visualizações de Dados - Tabelas:
Crie uma tabela interativa que exiba os dados carregados e permita ao usuário ordenar e filtrar as colunas diretamente pela interface.

In [53]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time
from st_aggrid import AgGrid, GridOptionsBuilder

def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()
            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filtra_df(df):
    if df is not None:
        
        #Inicializo o session_state
        if 'filter_cont' not in st.session_state:
            st.session_state.filter_cont = 'Todos'
        if 'list_colunas' not in st.session_state:
            st.session_state.list_colunas = df.columns.tolist()

        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Crio o filtro de continentes
        st.session_state.filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=list_cont.index(st.session_state.filter_cont))

        #Aplico a condição para só filtrar se houver seleção
        if st.session_state.filter_cont != 'Todos':
            df = df.loc[df['Continente'] == st.session_state.filter_cont]

        #Crio o filtro de colunas
        st.session_state.list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, st.session_state.list_colunas)

        #Aplico o filtro
        df_filt = df[st.session_state.list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()
    
        return df_filt

def exibe_tabelas(df_filt):
    st.write('## Tabela Comum')
    st.dataframe(df)

    st.write('## Tabela Interativa')
    if df_filt is not None:
        gb = GridOptionsBuilder.from_dataframe(df_filt)
        gb.configure_default_column(editable=False, groupable=True)
        gb.configure_selection('multiple', use_checkbox=True)
        gridOptions = gb.build()

        AgGrid(df_filt, gridOptions=gridOptions, enable_enterprise_modules=True)

################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

if files:
    df, csv = tratamento(files)
    df_filt = filtra_df(df)
    exibe_tabelas(df_filt)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")

end = time.time()
st.write(f'Tempo de execução: {end - start:.2f} segundos')
    

Overwriting app.py


### 10. Criar Visualizações de Dados - Gráficos Simples:
Desenvolva gráficos simples (como barras, linhas, e pie charts) para visualização dos dados carregados, utilizando o Streamlit.

In [81]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time
from st_aggrid import AgGrid, GridOptionsBuilder
import plotly.express as px


def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()
            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filtra_df(df):
    if df is not None:
        
        #Inicializo o session_state
        if 'filter_cont' not in st.session_state:
            st.session_state.filter_cont = 'Todos'
        if 'list_colunas' not in st.session_state:
            st.session_state.list_colunas = df.columns.tolist()

        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Crio o filtro de continentes
        st.session_state.filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=list_cont.index(st.session_state.filter_cont))

        #Aplico a condição para só filtrar se houver seleção
        if st.session_state.filter_cont != 'Todos':
            df = df.loc[df['Continente'] == st.session_state.filter_cont]

        #Crio o filtro de colunas
        st.session_state.list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, st.session_state.list_colunas)

        #Aplico o filtro
        df_filt = df[st.session_state.list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()
    
        return df_filt

def exibe_tabelas(df_filt):
    st.write('## Tabela Comum')
    st.dataframe(df)

    st.write('## Tabela Interativa')
    if df_filt is not None:
        gb = GridOptionsBuilder.from_dataframe(df_filt)
        gb.configure_default_column(editable=False, groupable=True)
        gb.configure_selection('multiple', use_checkbox=True)
        gridOptions = gb.build()

        AgGrid(df_filt, gridOptions=gridOptions, enable_enterprise_modules=True)

def graf_simples(df):
    # Criando o gráfico de pizza
    fig = px.pie(
        df,
        values='Qtd',
        names='Continente',
        title='Quantidade de Turistas por Continente',
        labels={'Qtd Tutistas': 'Qtd'},
        hole=False
    )

    fig.update_traces(textinfo='label+percent', textfont_size=10)

    st.plotly_chart(fig)

    #Criando Gráfico de Barras
    fig = px.bar(
        df,
        x='Pais',
        y='Qtd',
        title='Quantidade de Turistas por Pais',
        labels={'Qtd Tutistas': 'Qtd'}
    )

    st.plotly_chart(fig)


################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

if files:
    df, csv = tratamento(files)
    df_filt = filtra_df(df)
    exibe_tabelas(df_filt)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")
    graf_simples(df)
end = time.time()
st.write(f'Tempo de execução: {end - start:.2f} segundos')

Overwriting app.py


### 11. Criar Visualizações de Dados - Gráficos Avançados:
Adicione gráficos mais avançados (como histograma ou scatter plot) para fornecer insights mais profundos sobre os dados.

In [85]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time
from st_aggrid import AgGrid, GridOptionsBuilder
import plotly.express as px


def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()
            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filtra_df(df):
    if df is not None:
        
        #Inicializo o session_state
        if 'filter_cont' not in st.session_state:
            st.session_state.filter_cont = 'Todos'
        if 'list_colunas' not in st.session_state:
            st.session_state.list_colunas = df.columns.tolist()

        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Crio o filtro de continentes
        st.session_state.filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=list_cont.index(st.session_state.filter_cont))

        #Aplico a condição para só filtrar se houver seleção
        if st.session_state.filter_cont != 'Todos':
            df = df.loc[df['Continente'] == st.session_state.filter_cont]

        #Crio o filtro de colunas
        st.session_state.list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, st.session_state.list_colunas)

        #Aplico o filtro
        df_filt = df[st.session_state.list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()
    
        return df_filt

def exibe_tabelas(df_filt):
    st.write('## Tabela Comum')
    st.dataframe(df)

    st.write('## Tabela Interativa')
    if df_filt is not None:
        gb = GridOptionsBuilder.from_dataframe(df_filt)
        gb.configure_default_column(editable=False, groupable=True)
        gb.configure_selection('multiple', use_checkbox=True)
        gridOptions = gb.build()

        AgGrid(df_filt, gridOptions=gridOptions, enable_enterprise_modules=True)

def graficos(df):
    # Criando o gráfico de pizza
    fig = px.pie(
        df,
        values='Qtd',
        names='Continente',
        title='Quantidade de Turistas por Continente',
        labels={'Qtd Tutistas': 'Qtd'},
        hole=False
    )

    fig.update_traces(textinfo='label+percent', textfont_size=10)

    st.plotly_chart(fig)

    #Criando Gráfico de Barras
    fig = px.bar(
        df,
        x='Pais',
        y='Qtd',
        title='Quantidade de Turistas por Pais',
        labels={'Qtd Tutistas': 'Qtd'}
    )

    st.plotly_chart(fig)

    #Criando histrograma
    fig = px.histogram(
        df,
        x='Qtd',
        title='Distribuição da Qtd de Turistas',
        labels={'Qtd': 'Quantidade de Turistas'},
        nbins=20
    )

    st.plotly_chart(fig)

################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

if files:
    df, csv = tratamento(files)
    df_filt = filtra_df(df)
    exibe_tabelas(df_filt)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")
    graficos(df)
end = time.time()
st.write(f'Tempo de execução: {end - start:.2f} segundos')

Overwriting app.py


### 12. Exibir Métricas Básicas:
Implemente a exibição de métricas básicas (como contagem de registros, médias, somas) diretamente na interface para fornecer um resumo rápido dos dados carregados.

In [100]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time
from st_aggrid import AgGrid, GridOptionsBuilder
import plotly.express as px


def barra_lateral():
    st.sidebar.header('Filtros')
    #Adicionando o Color Picker
    cor_fundo = st.sidebar.color_picker("Cor de fundo", "#FFFFFF")
    cor_fonte = st.sidebar.color_picker("Cor da fonte", "#000000")  

    #Alterando o estilo da classe stApp
    st.markdown(f"<style> .stApp {{background-color: {cor_fundo}; color:{cor_fonte};}}   </style>", unsafe_allow_html=True)

def upload():
    files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)
    return files

@st.cache_data
def tratamento(files):
    
    df = pd.DataFrame()
    list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']
    
    if files != []:
        #Adicionando o Spinner
        with st.spinner('Upload em andamento...'):
            time.sleep(2)
            st.success("Upload concluído!")

        for file in files:
            data = pd.read_csv(file)
            #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
            line = data.iloc[1][0]
            #Utilizo Regex para extrair o ano
            Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   

            #Depois trato o restante do arquivo para ficar no formato desejado    
            data = data.iloc[4:]
            data.iloc[0][0] = 'Pais'
            data.columns = data.iloc[0]
            data = data.drop(data.index[0])
            data = data.dropna()

            #Retiro a coluna e a linha de total
            data = data[data['Pais'] != 'Total']
            data = data.drop(columns='Total')
            data['Ano'] = Ano

            #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
            data['Continente'] = data['Pais'].apply(
                lambda line: next((pais for pais in list_continentes if pais in line), None)
            )
            data['Continente'] = data['Continente'].fillna(method='ffill')

            #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
            data = data[~data['Pais'].isin(list_continentes)]

            data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

            #Convertendo o campo Qtd para número
            data['Qtd'] = data['Qtd'].str.replace(' ', '')
            data['Qtd'] = pd.to_numeric(data['Qtd'], errors='coerce')
            
            #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
            df = pd.concat([df, data])

            #Converto para CSV
            csv = df.to_csv().encode("utf-8")

        return df, csv

def filtra_df(df):
    if df is not None:
        
        #Inicializo o session_state
        if 'filter_cont' not in st.session_state:
            st.session_state.filter_cont = 'Todos'
        if 'list_colunas' not in st.session_state:
            st.session_state.list_colunas = df.columns.tolist()

        #Crio a lista de continentes
        list_cont = df['Continente'].drop_duplicates().tolist()
        list_cont.append('Todos')

        #Crio o filtro de continentes
        st.session_state.filter_cont = st.sidebar.selectbox('Filtrar Continentes', list_cont, index=list_cont.index(st.session_state.filter_cont))

        #Aplico a condição para só filtrar se houver seleção
        if st.session_state.filter_cont != 'Todos':
            df = df.loc[df['Continente'] == st.session_state.filter_cont]

        df_filt = df.copy()
        #Crio o filtro de colunas
        st.session_state.list_colunas = st.sidebar.multiselect('Selecione as colunas', df.columns, st.session_state.list_colunas)

        #Aplico o filtro
        df_filt_tab = df[st.session_state.list_colunas]

        #Adicionando a barra de progresso
        my_bar = st.progress(0, text="Carregando a Visualização...")
        for percent_complete in range(100):
            time.sleep(0.001)
            my_bar.progress(percent_complete + 1, text="Carregando a Visualização...")
        time.sleep(0.5)
        my_bar.empty()
    
        return df_filt, df_filt_tab

def exibe_tabelas(df):
    st.write('## Tabela Comum')
    st.dataframe(df)

    st.write('## Tabela Interativa')
    if df is not None:
        gb = GridOptionsBuilder.from_dataframe(df)
        gb.configure_default_column(editable=False, groupable=True)
        gb.configure_selection('multiple', use_checkbox=True)
        gridOptions = gb.build()

        AgGrid(df, gridOptions=gridOptions, enable_enterprise_modules=True)

def graficos(df):
    # Criando o gráfico de pizza
    fig = px.pie(
        df,
        values='Qtd',
        names='Continente',
        title='Quantidade de Turistas por Continente',
        labels={'Qtd Tutistas': 'Qtd'},
        hole=False
    )

    fig.update_traces(textinfo='label+percent', textfont_size=10)

    st.plotly_chart(fig)

    #Criando Gráfico de Barras
    fig = px.bar(
        df,
        x='Pais',
        y='Qtd',
        title='Quantidade de Turistas por Pais',
        labels={'Qtd Tutistas': 'Qtd'}
    )

    st.plotly_chart(fig)

    #Criando histrograma
    fig = px.histogram(
        df,
        x='Qtd',
        title='Distribuição da Qtd de Turistas',
        labels={'Qtd': 'Quantidade de Turistas'},
        nbins=20
    )

    st.plotly_chart(fig)

def metricas(df):
    #Qtd linhas
    qtd_linhas = df.shape[0]
    
    #Média de passageiros
    media_psg = round(df['Qtd'].mean(),0)

    #Total de passageiros
    total_psg = df['Qtd'].sum()

    # Exibindo as métricas
    a,b,c = st.columns(3)

    with a:
        st.metric(label="Qtd Registros", value=qtd_linhas)
    with b:
        st.metric(label="Média Passageiros por Registro", value=media_psg)
    with c:
        st.metric(label="Total de Passageiros", value=total_psg)

################### INICIANDO O APP ######################

barra_lateral()
files = upload()

start = time.time()

st.write('# Rodrigo_Mesquita_DR1_TP3')
st.write('## Desenvolvimento Front-End com Python (com Streamlit) [24E3_1]')
st.write('Notebook disponível em: https://github.com/rodrigo1992-cmyk/DR1_TP3')
st.write('Para testar o aplicativo, realizar upload dos arquivos disponíveis em: https://github.com/rodrigo1992-cmyk/DR1_TP3/tree/main/Data')


if files:
    df, csv = tratamento(files)
    df_filt, df_filt_tab = filtra_df(df)
    metricas(df_filt)
    graficos(df_filt)
    exibe_tabelas(df_filt_tab)
    st.download_button(label="Baixar Dados", data=csv, file_name="base.csv", mime="text/csv")
end = time.time()

st.write(f'Tempo de execução: {end - start:.2f} segundos')

Overwriting app.py
